##### 필요 Import 

In [ ]:
import pandas as pd
import deepl

-----

##### 데이터 로드 및 번역 API 설정

In [ ]:
DATA_DIR = '/data/ephemeral/home/level2-nlp-generationfornlp-nlp-14-lv3/data/'
df = pd.read_csv(DATA_DIR + 'train.csv')

In [ ]:
auth_key = "API-KEY"
translator = deepl.Translator(auth_key)

-----
##### 역번역 함수

In [ ]:
def back_translate_question(question, change_lang):
    try:
        # 한국어 -> 영어로 번역
        translated = translator.translate_text(question, target_lang=change_lang, source_lang='KO')
        # 영어 -> 한국어로 역번역
        back_translated = translator.translate_text(translated.text, target_lang='KO')
        
        print(change_lang, "로 번역되는 중...")
        print("원래 문장: ", question)
        print("번역된 문장: ", back_translated)
        
        return back_translated.text, translated.text
    
    except Exception as e:
        print(f"Error translating question: {e}")
        return "", ""

-----
##### 역번역 진행

In [ ]:
# 번역된 문제와 BLEU 점수를 저장할 리스트
back_translations_en = []
back_translations_jp = []

In [ ]:
for idx, problem in df['problems'].items():
    try:
        # 'problems' 열의 문제는 문자열 형태의 딕셔너리로 가정
        problem_dict = eval(problem)
        question_korean = problem_dict.get('question', '')
        
        # 질문을 영어와 일본어로 번역 (DeepL API)
        back_translated_en, translated_en = back_translate_question(question_korean, "EN-US")
        back_translated_jp, translated_jp = back_translate_question(question_korean, "JA")
        
        # 번역된 질문을 리스트에 추가
        back_translations_en.append(back_translated_en)
        back_translations_jp.append(back_translated_jp)
    
    except Exception as e:
        print(f"Error processing row {idx}: {e}")
        back_translations_en.append("")
        back_translations_jp.append("")

-----
##### 역번역 데이터 백업

In [ ]:
bt_en = pd.DataFrame({
    'problems': back_translations_en
})
output_file = 'back_translations_en.csv'
bt_en.to_csv(DATA_DIR + output_file, index=False)

bt_jp = pd.DataFrame({
    'problems': back_translations_jp
})
output_file = 'back_translations_jp.csv'
bt_jp.to_csv(DATA_DIR + output_file, index=False)

-----

#### 데이터 결합
##### 결합을 진행하기 전에 반드시 **update_format**을 실행시키고, 할 것!!!

In [ ]:
data1 = pd.read_csv(DATA_DIR + 'train.csv')
data2 = pd.read_csv(DATA_DIR + 'train_bt_en.csv')
data3 = pd.read_csv(DATA_DIR + 'train_bt_jp.csv')

data4 = pd.concat([data1, data2, data3], axis=0)

In [ ]:
output_file = 'train_problems_bt.csv'
data4.to_csv(DATA_DIR + output_file, index=False)